In [2]:
import pandas as pd
import numpy as np
import pydicom
import os

sub_types = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']


base_dir = '/home/wkeeley/Desktop/Brain-Hemorrhaging'
image_dir = os.path.join(base_dir, 'Images')
train_dir = os.path.join(image_dir, 'Train_Images')
val_dir = os.path.join(image_dir, 'Validation_Images')

dcm_dir = os.path.join(base_dir, 'stage_1_train_images')
dcm_test_dir = os.path.join(base_dir, 'stage_1_test_images')

sub_train_dirs = []
sub_val_dirs = []

for i in sub_types:
    sub_train_dirs.append(os.path.join(train_dir, i))
    sub_val_dirs.append(os.path.join(val_dir, i))



In [9]:
#Setup file hierarchy
for i in [ base_dir, image_dir, train_dir, val_dir]:
    try:
        os.mkdir(i)
    except FileExistsError:
        pass

for i in sub_train_dirs:
    try:
        os.mkdir(i)
        os.mkdir(os.path.join(i, 'positive'))
        os.mkdir(os.path.join(i, 'negative'))
    except FileExistsError:
        pass

for i in sub_val_dirs:
    try:
        os.mkdir(i)
    except FileExistsError:
        pass
    

In [ ]:
data = pd.read_csv('stage_1_train.csv')

In [ ]:
#from See like a radiologist with systematic windowing
def window_image(img, window_center, window_width, intercept, slope):
    img = (img*slope + intercept)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    
    img = (img - img_min) / (img_max - img_min)
    return img

def get_windowing(data):
    dicom_fields = [data[('0028', '1050')].value, data[('0028', '1051')].value, 
                    data[('0028', '1052')].value, data[('0028', '1053')].value]
    int_fields = []
    for x in dicom_fields:
        if type(x) == pydicom.multival.MultiValue:
            int_fields.append(int(x[0]))
        else:
            int_fields.append(int(x))
    return int_fields
    
        

In [ ]:
ids = []

for i in range(len(data)):
    img_id = data.iloc[i, 0][12]


In [ ]:
import cv2

for i in ids:
    file = os.path.join(, img_id + '.dcm')
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing_data(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    for i in sub_types:
        if i == 'epidural':
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            if data.iloc[i, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'positive'), img_id + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'negative'), img_id + '.jpg')
            
            cv2.imwrite(file_path, img)

        elif i == 'intraparechymal':
            img = window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[i, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[1], 'positive'), img_id + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[1], 'negative'), img_id + '.jpg')
            cv2.imwrite(file_path, img)   
        elif i == 'intraventricular':
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[i, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[2], 'positive'), img_id + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[2], 'negative'), img_id + '.jpg')
            cv2.imwrite(file_path, img)
        elif i == 'subarachnoid':
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[i, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[3], 'positive'), img_id + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[3], 'negative'), img_id + '.jpg')
            cv2.imwrite(file_path, img)
        elif i == 'subdural':
            img = window_image(image, 80, 200, intercept, slope) #subdural window
            if data.iloc[i, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[4], 'positive'), img_id + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[4], 'negative'), img_id + '.jpg')
            cv2.imwrite(file_path, img)

In [ ]:
#epidural model
from keras import layers
from keras import models

